In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics
from flask import Flask, request, render_template,jsonify
from flask_cors import CORS
import re
import math
from utility.data_cleaning import *
from utility.data_preprocessing import *
from utility.model_building import *
from config.configuration import *


app = Flask(__name__)


#import pickle folder of model, encoder and mode values of cat columns 

with open('./data/training/output/lgb_model_20231206.pkl', 'rb') as file:
    model=pickle.load(file)
with open('./data/training/output/encoder_20231206.pkl', 'rb') as file:
    label_encoders=pickle.load(file)
modes_dict_path = './data/training/output/mode_values_dict_20231206.pkl'
with open(modes_dict_path, 'rb') as file:
    modes_dict = pickle.load(file)
print(modes_dict)
    
@app.route("/predict_api", methods=['POST'])
def RiskPrediction():
    json_ = request.json
    df= pd.DataFrame([json_])
    display(df)
    
    
    df.replace('',np.nan,inplace=True)
    
    
# check if the col of df presents in modes_dict, and if present then it replace the null value of cols with the mode values of this col.
    
    for col in df.columns: 
        if col in modes_dict:
            df[col]=df[col].fillna(modes_dict[col])
            
    
    display(df)
    
    df_clean = clean_data(df)
    display(df_clean)
     
#to get information about the classes of each col  
    for col, encoder in label_encoders.items():
        print(f"Column: {col}")
        print(f"Classes: {encoder.classes_}")
        print("------")
        
    for col in label_encoders:
        df_clean[col] = label_encoders[col].transform(df_clean[col])
        
       
        
    
    predicted_loss_amount= model.predict(df_clean)
    output = predicted_loss_amount[0]
    
    #print(output)
    #display(df_clean)
    return jsonify({"prediction":output})
if __name__ == '__main__':
    #app.run()
    app.run(host='0.0.0.0', port=80)

{'Jurisdiction': 'Pennsylvania', 'CoverageType': 'BOP-Building', 'Deductible': 'FixedDollar - 2500.00', 'Limit': 'Blanket Group 1 Blanket - 48075046.00', 'InjuryCause': 'WATER', 'YearBuilt': 1900.0, 'Stories': 2.0, 'Comments': 'ClassCode:65146', 'ConstructionTypeDesc': 'Frame', 'ZipCode': '18102', 'E2Value': 955986.0, 'SquareFootage': 3489, 'BuildingLimit': 610526}
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:80
 * Running on http://192.168.1.179:80
Press CTRL+C to quit


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,Maryland,BOP-Building,,Standard - 7096952.00,WATER,2008,4,ClassCode:65147,,17319,8308610.34,35000,7096952


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,Maryland,BOP-Building,FixedDollar - 2500.00,Standard - 7096952.00,WATER,2008,4,ClassCode:65147,Frame,17319,8308610.34,35000,7096952


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,maryland,bop-building,fixeddollar - 2500.00,standard - 7096952.00,water,2008,4,classcode:65147,frame,17319,8308610.34,35000,7096952


...

  Jurisdiction  CoverageType InjuryCause  Stories        ClassCode  \
0     maryland  bop-building       water        4  classcode:65146   

  ConstructionTypeDesc ZipCode     E2Value  SquareFootage  BuildingLimit  \
0                frame   vinay  8308610.34          35000         610526   

  limit_type  Limit_val Deductible_type  Deductible_val  YearGap  
0   standard  7096952.0     fixeddollar            2500      123  


,Jurisdiction,CoverageType,InjuryCause,Stories,ClassCode,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit,limit_type,Limit_val,Deductible_type,Deductible_val,YearGap
0,maryland,bop-building,water,4,classcode:65146,frame,vinay,8308610.34,35000,610526,standard,7096952.0,fixeddollar,2500,123


[2023-12-06 13:55:03,192] ERROR in app: Exception on /predict_api [POST]
Traceback (most recent call last):
  File "C:\Users\SmrutisikhaJena\anaconda3\Lib\site-packages\sklearn\utils\_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SmrutisikhaJena\anaconda3\Lib\site-packages\sklearn\utils\_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SmrutisikhaJena\anaconda3\Lib\site-packages\sklearn\utils\_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "C:\Users\SmrutisikhaJena\anaconda3\Lib\site-packages\sklearn\utils\_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'vinay'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SmrutisikhaJena\

Column: Jurisdiction
Classes: ['delaware' 'dist. of columbia' 'maryland' 'north carolina' 'ohio'
 'pennsylvania' 'virginia']
------
Column: CoverageType
Classes: ['bop-building']
------
Column: InjuryCause
Classes: ['electric current' 'fire' 'freezing' 'hail' 'other perils'
 'vandalism & malicious mischief' 'vehicles' 'water' 'wind']
------
Column: ClassCode
Classes: ['classcode:09061' 'classcode:09651' 'classcode:0966109661'
 'classcode:59997' 'classcode:59999' 'classcode:63611' 'classcode:63981'
 'classcode:65132' 'classcode:65141' 'classcode:65142' 'classcode:65144'
 'classcode:65145' 'classcode:6514565145' 'classcode:65146'
 'classcode:65147' 'classcode:65171' 'classcode:69145' 'classcode:75521']
------
Column: ConstructionTypeDesc
Classes: ['frame' 'joistedmasonry' 'masonrynoncombustible']
------
Column: ZipCode
Classes: ['15045' '15108' '15120' '15133' '15144' '15145' '15206' '15210' '15212'
 '15217' '15218' '15232' '15236' '15301' '15456' '15601' '15620' '15644'
 '15661' '15714'

127.0.0.1 - - [06/Dec/2023 13:55:03] "POST /predict_api HTTP/1.1" 500 -


In [1]:
import pandas as pd
data= {
    "Jurisdiction":"Delaware",
    "CoverageType":"BOP-Building",
    "Deductible":"FixedDollar - 1000.00",
    "Limit":"Standard - 217000.00",
    "InjuryCause":"WIND",
    "YearBuilt":"1958",
    "Stories":"1",
    "Comments":"ClassCode:65141",
    "ConstructionTypeDesc":"Frame",
    "ZipCode":"19973",
    "E2Value":"217236",
    "SquareFootage":"1480",
    "BuildingLimit":"217000"
}
df=pd.DataFrame([data])
display(df)

,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,Delaware,BOP-Building,FixedDollar - 1000.00,Standard - 217000.00,WIND,1958,1,ClassCode:65141,Frame,19973,217236,1480,217000
